In [ ]:
import subprocess
from pathos.multiprocessing import ProcessPool
import os
import random
import requests

if not os.path.exists("raw_traces"):
    os.mkdir("raw_traces")
else:
    # delete all files in raw_traces
    for file in os.listdir("raw_traces"):
        os.remove(os.path.join("raw_traces", file))

In [ ]:
num_topics = 16

In [ ]:
topics = [f'/eth2/fulu_fork_digest/data_column_sidecar_{i}/ssz_snappy' for i in range(num_topics)]

In [ ]:
ips = []
with open("ips.txt", "r") as f:
    for line in f:
        ips.append(line.strip().split(":")[0])

nodes_down = []

for ip in ips: 
    status = requests.get(f"http://{ip}:9090/api/v1/health", timeout=10)
    print(status.json())
    if status.json()['status'] != 'ok':
        nodes_down.append(ip)

print("Nodes up: ", len(ips) - len(nodes_down))
print("Nodes down: ", len(nodes_down))

if len(nodes_down) > 0:
    print("Nodes down: ", nodes_down)



In [ ]:
subscribe_processes = []  # Keep track of all started processes
trace_files = []

for i,topic in enumerate(topics):
    trace_file = f"raw_traces/trace-data-col-{i}.tsv"
    trace_files.append(trace_file)
    # select a random node to publish
    proc = subprocess.Popen([
        "./p2p-multi-subscribe",
        "-topic", topic,
        "-ipfile", "ips.txt",
        "-output-trace", trace_file,
    ])
    subscribe_processes.append(proc)

print(f"Started {len(subscribe_processes)} subscriber processes")



In [ ]:
cmd = [
    "./batch-publish",
    "-addr", f"{ips[0]}:33212",
    "-topics", ",".join(topics),
    "-msg", "16384",
    "-sleep", "12s",
    "-num_batches","1",
]

subprocess.run(cmd)

In [ ]:
merged_traces_file = "merged_traces.tsv"
with open(merged_traces_file, "w") as f:
    for trace_file in trace_files:
        if os.path.exists(trace_file):
            with open(trace_file, "r") as t:
                
                f.write(t.read())
        else:
            print(f"Warning: trace file {trace_file} not found. Skipping.")